<div class="alert alert-block alert-info">paraprobe-toolbox.</div>

# An exhaustive example with an ODS steel

**Markus Kühbach** (Department of Physics, Humboldt-Universität zu Berlin)<br>
Jing Wang, Daniel Schreiber (both at their times with the Pacific Northwest National Laboratory)

## Contextualization / Problem statement
***

This is a tutorial on how to use the paraprobe-toolbox for processing a datasets with procedures which<br>
are better aligned with the aims of the FAIR data stewardship principles.<br>
<a href="https://doi.org/10.1017/S1431927618015386">The dataset is an ODS steel with Y-containing titanium oxid dispersoids by J. Wang and coworkers</a>.<br>


The analyses here show how tools and functionalities of the paraprobe-toolbox can be combined, results transferred between these tools,<br>
and NeXus and HDF5 used in concert, to analyze the authors' dataset. Specifically, here we show how precipitates can be reconstructed<br>
using iso-surfaces. Furthermore it will be shown how to characterize the objects individually.

What makes the paraprobe-toolbox approach special is not only its efficiency and documentation capabilities per se but<br> 
how these capabilities enable systematic studies of how the analysis results depend on the parameterization of the<br>
computational geometry methods that are typically used for analyzing atom probe data.<br>
Such parameter sensitivity studies are useful because techniques like delocalization, iso-surfaces,<br>
and composition analyses can show significant variation based on which regions-of-interest (ROIs) are chosen.<br>

The following analysis has the following workflow:<br>
1. Pre-processing of an existent dataset, reconstructed and ranged, creating a model<br>
   for the edge of the dataset (reconstructed volume) for handling and correcting for<br>
   finite size (edge) effects, distancing of all ions to this edge, and tessellating<br>
   the entire dataset using a Voronoi tessellation.<br>
2. Perform multiple analyses of objects, here exemplified by objects containing Y+Ti+O, i.e.<br>
   ODS steel-relevant yttria dispersoids likely, using different iso-composition values.<br>
   The objects will be characterized with commonly used descriptors (volume fraction, number<br>
   composition, size and shape distribution). In the process, triangulated surface meshes<br>
   for each objects will be created. These meshes are the input for the intersection analyses.
3. Portions of iso-surface meshes for arbitrary datasets have not necessarily all sections<br>
   which are watertight. This example will also show how these incomplete/patchy portions<br>
   of the iso-surface can be post-processed by replacing them with so-called proxies<br>
   to obtain models of watertight objects.

The example shows how all tools, i.e. jupyter-lab, h5web, and the C++ and Python tools of the paraprobe-toolbox<br>
work together to programmatically instruct, execute, and document the above-described analyses.<br>

## Get the toolbox ready
***

### Specify the location of the paraprobe-toolbox.

In [ ]:
# specify where the toolbox is installed
# MYTOOLBOXPATH='<<YOUR PREFIX>>/paraprobe-toolbox'
MYTOOLBOXPATH='../../../'
from jupyterlab_h5web import H5Web
from IPython.display import Image
import sys, os
import h5py, ase
import numpy as np
sys.path.append(MYTOOLBOXPATH + '/code')
sys.path.append(MYTOOLBOXPATH + '/code/paraprobe-parmsetup/src/python')
sys.path.append(MYTOOLBOXPATH + '/code/paraprobe-transcoder/src/python')
sys.path.append(MYTOOLBOXPATH + '/code/paraprobe-autoreporter/src/python')
print('Current working directory is')
print(os.getcwd())
# simplify startup in the future

### Load the tools of the toolbox.

In [ ]:
# Python parmsetup utility tools which create configuration files
from tools.transcoder_guru import ParmsetupTranscoder, TranscodingTask
from tools.ranger_guru import ParmsetupRanger, ApplyExistentRanging, MolecularIonSearch
from tools.surfacer_guru import ParmsetupSurfacer, SurfaceMeshingTask
from tools.distancer_guru import ParmsetupDistancer, PointToTriangleSoupDistancing
from tools.tessellator_guru import ParmsetupTessellator, TessellationTask
from tools.spatstat_guru import ParmsetupSpatstat, SpatstatTask
from tools.nanochem_guru import ParmsetupNanochem, NanochemTask, Delocalization, InterfaceMeshing, CompositionAnalysis
from tools.intersector_guru import ParmsetupIntersector, VolumeFeatureSubSet, VolumeFeatureSet, VolumeVolumeTask

# Python transcoder utility tool which imports atom probe file formats
from paraprobe_transcoder import ParaprobeTranscoder
from tools.utils.numerics import EPSILON, get_file_size
from tools.utils.primscontinuum import RoiRotatedCuboid, RoiRotatedCylinder, RoiSphere
# C/C++ tools of the toolbox
# you can use the path in the respective paraprobe-<<toolname>>/build/paraprobe_<<toolname>>
# Python utility tools for examples of standardized post-processing and visualization
from wizard.ranger_report import AutoReporterRanger
from wizard.surfacer_report import AutoReporterSurfacer
from wizard.distancer_report import AutoReporterDistancer
from wizard.tessellator_report import AutoReporterTessellator
from wizard.spatstat_report import AutoReporterSpatstat
from wizard.nanochem_report import AutoReporterNanochem
# from wizard.intersector_report import AutoReporterIntersector

To learn how to handle and work with iontypes in paraprobe please inspect the specific tutorials.

# 1. Pre-processing
***

### Specify the location(s) of the your dataset(s).

In [ ]:
# specify the location where you have your data on the system
# testing without NeXus
MY_MEASURED_DATA_PATH = MYTOOLBOXPATH + '/teaching/example_data/usa_portland_wang'
MY_PROCESSED_DATA_PATH = MYTOOLBOXPATH + '/teaching/example_analyses/usa_portland_wang'
# testing with NeXus
# MY_MEASURED_DATA_PATH = MYTOOLBOXPATH + '/teaching/example_data/ger_berlin_kuehbach_fairmat'

# specify disjoint identifier with which all config and results files for this analysis will be tagged.
jobids = [636502001]
JOBIDS = []
for jobid in jobids:
    assert isinstance(jobid, int), 'identifier needs to be an unsigned integer !'
    assert jobid != 0, 'identifier must not be 0 !'
    assert jobid <= np.iinfo(np.uint32).max, 'identifier needs to be on interval [1, 4294967295]'
    JOBIDS.append(str(jobid))
print(JOBIDS)

## Import your data (from e.g. IVAS/APSuite, community tool) into the paraprobe-toolbox.

In [ ]:
# configure the paraprobe-transcoder tool
TRANSCODER_CONFIG = {}
RECONSTRUCTION_AND_RANGING = {}
RECONSTRUCTION_AND_RANGING[jobids[0]] = ('R31_06365-v02.pos', 'R31_06365-v02.rrng')

for jobid in jobids:
    JOBID = str(jobid)
    transcoder = ParmsetupTranscoder()

    TRANSCODER_CONFIG[jobid] = transcoder.load_reconstruction_and_ranging(
        working_directory='',
        reconstructed_dataset=MY_MEASURED_DATA_PATH + '/' + RECONSTRUCTION_AND_RANGING[jobid][0],
        ranging_definitions=MY_MEASURED_DATA_PATH + '/' + RECONSTRUCTION_AND_RANGING[jobid][1],
        jobid=jobid)

print(TRANSCODER_CONFIG)

In [ ]:
# inspect config file if desired
H5Web(TRANSCODER_CONFIG[jobid])

In [ ]:
# execute paraprobe-transcoder Python tool
TRANSCODER_RESULTS = {}
for jobid in jobids:
    transcoder = ParaprobeTranscoder(TRANSCODER_CONFIG[jobid])
    TRANSCODER_RESULTS[jobid] = transcoder.execute()
print(TRANSCODER_RESULTS)

In [ ]:
# inspect paraprobe-transcoder results
get_file_size(TRANSCODER_RESULTS[jobid])
H5Web(TRANSCODER_RESULTS[jobid])

In [ ]:
# perform additional customized data analyses here if desired

## Apply existent ranging definitions.

In [ ]:
# configure paraprobe-ranger
RANGER_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    ranger = ParmsetupRanger()
    RANGER_CONFIG[jobid] = ranger.apply_existent_ranging(
        working_directory=MY_PROCESSED_DATA_PATH,
        transcoder_config_sim_id=jobid,
        transcoder_results_sim_id=jobid,
        ranger_results_sim_id=jobid)
print(RANGER_CONFIG)

In [ ]:
# inspect config file if desired
H5Web(RANGER_CONFIG[jobid])

In [ ]:
# execute paraprobe-ranger C/C++ tool
RANGER_RESULTS = {}
for jobid in jobids:
    JOBID = str(jobid)
    CONFIG = RANGER_CONFIG[jobid]
    ! export OMP_NUM_THREADS=4 && mpiexec -n 1 ./../../../code/paraprobe_ranger $jobid $CONFIG 1>{'PARAPROBE.Ranger.StdOutLog.SimID.' + JOBID + '.txt'} 2>{'PARAPROBE.Ranger.StdErrLog.SimID.' + JOBID + '.txt'}
    RANGER_RESULTS[jobid] = 'PARAPROBE.Ranger.Results.SimID.' + JOBID + '.nxs'
print(RANGER_RESULTS)

In [ ]:
# inspect paraprobe-ranger results
get_file_size(RANGER_RESULTS[jobid])
H5Web(RANGER_RESULTS[jobid])

In [ ]:
# perform additional customized data analyses here if desired, either through writing analysis code directly in the notebook

See how results can be post-processed specific for each tool using the convenience reporting and visualization Python functions offer through paraprobe-autoreporter.

In [ ]:
# or through using predefined analyses and/or use them as examples and customize them for your needs
# use corporate design/preconfigured analyses from paraprobe-autoreporter
for jobid in jobids:
    JOBID = str(jobid)
    print(JOBID)
    ranger_report = AutoReporterRanger(RANGER_RESULTS[jobid], JOBID)
    ranger_report.get_summary()

## Create a triangle surface mesh model for the edge of the dataset.

In [ ]:
# configure paraprobe-surfacer
SURFACER_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    surfacer = ParmsetupSurfacer()
    SURFACER_CONFIG[jobid] = surfacer.compute_convex_hull_edge_model(
        working_directory=MY_PROCESSED_DATA_PATH,
        transcoder_config_sim_id=jobid,
        transcoder_results_sim_id=jobid,
        ranger_results_sim_id=jobid,
        surfacer_results_sim_id=jobid)
print(SURFACER_CONFIG)

In [ ]:
# inspect config if necessary
H5Web(SURFACER_CONFIG[jobid])

In [ ]:
# execute paraprobe-surfacer C/C++ tool
SURFACER_RESULTS = {}
for jobid in jobids:
    JOBID = str(jobid)
    CONFIG = SURFACER_CONFIG[jobid]
    ! export OMP_NUM_THREADS=1 && mpiexec -n 1 ./../../../code/paraprobe_surfacer $jobid $CONFIG 1>{'PARAPROBE.Surfacer.StdOutLog.SimID.' + JOBID + '.txt'} 2>{'PARAPROBE.Surfacer.StdErrLog.SimID.' + JOBID + '.txt'}
    SURFACER_RESULTS[jobid] = 'PARAPROBE.Surfacer.Results.SimID.' + JOBID + '.nxs'
print(SURFACER_RESULTS)

In [ ]:
# inspect paraprobe-surfacer results
print(str(np.around(os.path.getsize(SURFACER_RESULTS[jobid])/1024/1024, decimals=3)) + ' MiB')
H5Web(SURFACER_RESULTS[jobid])

In [ ]:
# perform additional customized data analyses here if desired, either through writing analysis code directly in the notebook

## Compute Euclidean distances of all ions to the edge of the dataset.

In [ ]:
# configure paraprobe-distancer
DISTANCER_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    distancer = ParmsetupDistancer()
    DISTANCER_CONFIG[jobid] = distancer.compute_ion_to_edge_model_distances(
        working_directory=MY_PROCESSED_DATA_PATH,
        transcoder_config_sim_id=jobid,
        transcoder_results_sim_id=jobid,
        ranger_results_sim_id=jobid,
        distancer_results_sim_id=jobid)
print(DISTANCER_CONFIG)

In [ ]:
# inspect config if desired
H5Web(DISTANCER_CONFIG[jobid])

In [ ]:
# execute paraprobe-distancer C/C++ tool
DISTANCER_RESULTS = {}
for jobid in jobids:
    JOBID = str(jobid)
    CONFIG = DISTANCER_CONFIG[jobid]
    ! export OMP_NUM_THREADS=4 && mpiexec -n 1 ./../../../code/paraprobe_distancer $jobid $CONFIG 1>{'PARAPROBE.Distancer.StdOutLog.SimID.' + JOBID + '.txt'} 2>{'PARAPROBE.Distancer.StdErrLog.SimID.' + JOBID + '.txt'}
    DISTANCER_RESULTS[jobid] = 'PARAPROBE.Distancer.Results.SimID.' + JOBID + '.nxs'
print(DISTANCER_RESULTS)

In [ ]:
# inspect config if desired
print(str(np.around(os.path.getsize(DISTANCER_RESULTS[jobid])/1024/1024, decimals=3)) + ' MiB')
H5Web(DISTANCER_RESULTS[jobid])

See how to post-process results from paraprobe-distancer using paraprobe-autoreporter.

In [ ]:
# additional corporate design/preconfigured analyses from paraprobe-autoreporter
DISTANCER_PLOT = {}
for jobid in jobids:
    JOBID = str(jobid)
    distancer_report = AutoReporterDistancer(DISTANCER_RESULTS[jobid], JOBID)
    DISTANCER_PLOT[jobid] = distancer_report.get_ion2mesh_distance_cdf(distancing_task_id=0)
print(DISTANCER_PLOT)

In [ ]:
Image(filename=DISTANCER_PLOT[jobid], width=500, height=500)

In [ ]:
# alternatively you can of course also post-process the results yourself and inspect beyond what paraprobe-autoreporter offers
# below is an example how to use Python for this task, functions similar as the one shown here is what paraprobe-autoreporter implements
h5r = h5py.File(DISTANCER_RESULTS[jobid], 'r')
d = np.asarray(h5r['/entry/process0/point_to_triangle_set/distance'][:])
threshold = 1.0 #nm
fraction = 100. * np.count_nonzero(np.where(d <= threshold)) / np.shape(d)[0]
print('Your chosen threshold distance to the edge of the dataset is ' + str(np.around(threshold, decimals=2)) + ' nm')
print('Fraction of ions closer than this threshold distance ' + str(np.around(fraction, decimals=2)) + ' at.-%')
print('Minimum distance is ' + str(np.min(d)) + ' nm')  #  often ions on the surface of e.g. a convex hull
print('Average distance is ' + str(np.mean(d)) + ' nm')
print('Median distance is ' + str(np.median(d)) + ' nm')
print('Maximum distance is ' + str(np.max(d)) + ' nm')
quantile = 0.01
print('Specific distance at the ' + str(np.around(quantile*100., decimals=3))
      + '% percentile is ' + str(np.around(np.quantile(d, quantile), decimals=3)) + ' nm')
quantile = 0.05
print('Specific distance at the ' + str(np.around(quantile*100., decimals=3))
      + '% percentile is ' + str(np.around(np.quantile(d, quantile), decimals=3)) + ' nm')
# print('Maybe we would like ask turn the question on quantiles around and find the quantile value?')

The idea of paraprobe-autoreporter is that you can use it to register frequently used scripts and allow for sharing of Python code.

## Tessellate the ion point cloud.

In [ ]:
# configure paraprobe-tessellator
TESSELLATOR_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    tessellator = ParmsetupTessellator()
    TESSELLATOR_CONFIG[jobid] = tessellator.compute_complete_voronoi_tessellation(
        working_directory=MY_PROCESSED_DATA_PATH,
        transcoder_config_sim_id=jobid,
        transcoder_results_sim_id=jobid,
        ranger_results_sim_id=jobid,
        distancer_results_sim_id=jobid,
        tessellator_results_sim_id=jobid)
print(TESSELLATOR_CONFIG)

In [ ]:
# inspect config if desired
H5Web(TESSELLATOR_CONFIG[jobid])

In [ ]:
# execute paraprobe-tessellator C/C++ tool
TESSELLATOR_RESULTS = {}
for jobid in jobids:
    JOBID = str(jobid)
    CONFIG = TESSELLATOR_CONFIG[jobid]
    ! export OMP_NUM_THREADS=4 && mpiexec -n 1 ./../../../code/paraprobe_tessellator $jobid $CONFIG 1>{'PARAPROBE.Tessellator.StdOutLog.SimID.' + JOBID + '.txt'} 2>{'PARAPROBE.Tessellator.StdErrLog.SimID.' + JOBID + '.txt'}
    TESSELLATOR_RESULTS[jobid] = 'PARAPROBE.Tessellator.Results.SimID.' + JOBID + '.nxs'
print(TESSELLATOR_RESULTS)

In [ ]:
# inspect paraprobe-tessellator results
print(str(np.around(os.path.getsize(TESSELLATOR_RESULTS[jobid])/1024/1024, decimals=3)) + ' MiB')
H5Web(TESSELLATOR_RESULTS[jobid])

See how to post-process paraprobe-tessellator results with paraprobe-autoreporter.

In [ ]:
# additional corporate design/preconfigured analyses from paraprobe-autoreporter
TESSELLATOR_PLOT = {}
for jobid in jobids:
    JOBID = str(jobid)
    tessellator_report = AutoReporterTessellator(TESSELLATOR_RESULTS[jobid], JOBID)
    tessellator_report.get_summary(tessellation_task_id=0)
    TESSELLATOR_PLOT[jobid] = tessellator_report.get_cell_volume_cdf(tessellation_task_id=0)
print(TESSELLATOR_PLOT)

In [ ]:
Image(filename=TESSELLATOR_PLOT[jobid], width=500, height=500)

In [ ]:
# alternatively you can of course also post-process the results yourself and inspect beyond what paraprobe-autoreporter offers

## Optionally compute spatial statistics.

In [ ]:
# get axis-aligned bounding box of the reconstructed volume
h5r = h5py.File('PARAPROBE.Transcoder.Results.SimID.636502001.nxs', 'r')
xyz = h5r['/entry/atom_probe/reconstruction/reconstructed_positions'][:, :]
h5r.close()
aabb = np.zeros([3, 2], np.float32)
center = [0., 0., 0.]  # np.zeros([3], np.float32)
for i in np.arange(0, 3):
    aabb[i, 0] = np.min(xyz[:, i])
    aabb[i, 1] = np.max(xyz[:, i])
    center[i] = 0.5 * (aabb[i, 0] + aabb[i, 1])
del xyz
print('aabb bounding box')
print(aabb)
print('aabb center')
print(center)
print(np.shape(center))

In [ ]:
# configure paraprobe-spatstat
SPATSTAT_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    spatstat = ParmsetupSpatstat()
    
    # define two tasks, first instantiate a task object
    task = SpatstatTask()
    task.load_reconstruction_and_ranging(
        ranging_applied=True,
        working_directory=MY_PROCESSED_DATA_PATH,
        transcoder_config_sim_id=jobid,
        transcoder_results_sim_id=jobid,
        ranger_results_sim_id=jobid)
    task.load_ion_to_edge_distances(
        file_name=MY_PROCESSED_DATA_PATH + '/PARAPROBE.Distancer.Results.SimID.' + JOBID + '.nxs',
        dataset_name='/entry/process0/point_to_triangle_set/distance',
        d_edge=0.160)
    task.load_ion_to_feature_distances(
        file_name=MY_PROCESSED_DATA_PATH + '/PARAPROBE.Distancer.Results.SimID.' + JOBID + '.nxs',
        dataset_name='/entry/process0/point_to_triangle_set/distance',
        d_feature=0.678)
    # from the 1% to the 5% of the signed distance distribution
    # this will place ROIs at all those (source) ions laying at least d_edge from the edge
    # given we also use the edge as the feature (simply for demonstration, one could also use something else)
    # do not place ROIs at all those (source) ions laying farther away than d_feature to the edge
    # here the feature is the edge, in this case we can compute spatial statistics with ions
    # within a specifically customized shell closer to the edge than feature distance but not placing ROIs the subset
    # of these ions which are closer to the edge than edge distance
    # in addition we do not need to analyze the entire dataset, but we can also restrict the analysis to a particular window
    # filter region of interest
    task.cfg_filter.add_spatial_filter(primitive_list=[RoiRotatedCuboid(center=center, boxdims=[10., 10., 40.])])
    # task.cfg_filter.add_evaporation_id_filter(lival=(0, 1, np.iinfo(np.uint32).max))
    # task.cfg_filter.add_iontyp_filter()
    # task.cfg_filter.add_hit_multiplicity_filter()
    task.randomize_ion_types(True)
    task.set_random_number_generator()

    # that is 2nm from the edge of the dataset but no ROIs will be placed at ions deeper embedded in the dataset
    # ions closer to the edge than 2nm can be target like ions within the interior, with details depending on rmax
    # task.ion_types_source(method='resolve_ion', symbol_lst=[['Y', 'O']], charge_lst=[+1])
    # task.ion_types_target(method='resolve_ion', symbol_lst=[['Y', 'O'], ['Cr']], charge_lst=[+1, +3])
    # task.ion_types_source(method='resolve_element', symbol_lst=[['Y', 'O']])
    # task.ion_types_target(method='resolve_element', symbol_lst=[['Cr']])
    # task.ion_types_source(method='resolve_unknown')
    # task.ion_types_target(method='resolve_unknown')
    task.ion_types_source(method='resolve_all')
    task.ion_types_target(method='resolve_all')
    
    # currently KNN and RDF are implemented as either or per task as for IUC09 this is the most relevant case
    task.set_knn(kth=1, binwidth=0.01, rmax=2.)
    spatstat.add_task(task)
    
    # # either create a second task object and change what you want or change a property of the last task
    # task.set_rdf(binwidth=0.01, rmax=2.)
    # spatstat.add_task(task)
    
    SPATSTAT_CONFIG[jobid] = spatstat.configure(jobid)  # , verbose=True)
print(SPATSTAT_CONFIG)

In [ ]:
# inspect paraprobe-spatstat config, if desired
H5Web(SPATSTAT_CONFIG[jobid])

In [ ]:
# execute paraprobe-spatstat C/C++ tool
SPATSTAT_RESULTS = {}
for jobid in jobids:
    JOBID = str(jobid)
    CONFIG = SPATSTAT_CONFIG[jobid]
    ! export OMP_NUM_THREADS=4 && mpiexec -n 1 ./../../../code/paraprobe_spatstat $jobid $CONFIG 1>{'PARAPROBE.Spatstat.StdoutLog.SimID.' + JOBID + '.txt'} 2>{'PARAPROBE.Spatstat.StderrLog.SimID.' + JOBID + '.txt'}
    SPATSTAT_RESULTS[jobid] = 'PARAPROBE.Spatstat.Results.SimID.' + JOBID + '.nxs'
print(SPATSTAT_RESULTS)

In [ ]:
# inspect paraprobe-spatstat results
print(str(np.around(os.path.getsize(SPATSTAT_RESULTS[jobid])/1024/1024, decimals=3)) + ' MiB')
H5Web(SPATSTAT_RESULTS[jobid])

See how to post-process paraprobe-spatstat results with paraprobe-autoreporter.

<div class="alert alert-block alert-warning">
For computing RDFs according to B. Gault et al. (https://dx.doi.org/10.1007/978-1-4614-3436-8)<br>
do not forget to define a proper value for $\frac{1}{\rho}$ the scaling density of the RDF.</div>

In [ ]:
# additional corporate design/preconfigured analyses from paraprobe-autoreporter
SPATSTAT_PLOT = {}
# for jobid in jobids:
# JOBID = str(jobid)
spatstat_report = AutoReporterSpatstat(SPATSTAT_RESULTS[jobid], JOBID)
spatstat_report.get_knn(spatstat_task_id=0)
# spatstat_report.get_rdf(spatstat_task_id=0, normalizer=1./1.) # dont forget to make a proper estimate for rho!
# print(SPATSTAT_PLOT)

In [ ]:
Image(filename='PARAPROBE.Spatstat.Results.SimID.636502001.nxs.TaskId.0.Knn..Pdf.png', width=500, height=500)
# Image(filename='PARAPROBE.Spatstat.Results.SimID.636502001.h5.TaskId.0.Rdf.png', width=500, height=500)

# 2. High-throughput characterization of Y+Ti+O rich objects using iso-composition surfaces
***

## Characterize Y+Ti+O rich objects

We use paraprobe-nanochem, the same tool that can also be used for high-throughput iso-surface based analyses, composition profiling, and iso-surface-based edge modelling.

In [ ]:
# configure paraprobe-nanochem
what_to_do = 'isosurface_scanning'  # high-throughput scanning of iso-surfaces and analyses of closed objects formed by the iso-surfaces, use this also as the first step for coprecipitation analyses
# what_to_do = 'isosurface_edge_model'  # iso-surface-based modeling of the edge of the dataset (total-atom-count based approach mentioned by M. K\"uhbach et al. npj Comp Mat, 2022)
# what_to_do = 'find_interface'  # PCA+DCOM for automated locating and meshing of interface patch triangulated surface mesh models
# what_to_do = 'rois_and_co'  # place ROIs automatically at given mesh (e.g. from a previous find_interface) analysis, get composition profiles enabling interfacial excess computation in python

NANOCHEM_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    
    nanochem = ParmsetupNanochem()
    if what_to_do == 'isosurface_scanning':
        dataset = NanochemTask()

        dataset.load_reconstruction_and_ranging(
            ranging_applied=True,
            working_directory=MY_PROCESSED_DATA_PATH,
            transcoder_config_sim_id=jobid,
            transcoder_results_sim_id=jobid,
            ranger_results_sim_id=jobid)
        
        dataset.load_edge_model(
            file_name='PARAPROBE.Surfacer.Results.SimID.' + JOBID + '.nxs',
            dataset_name_vertices='/entry/process0/point_set_wrapping0/alpha_complex/triangle_set/triangles/vertices',
            dataset_name_facet_indices='/entry/process0/point_set_wrapping0/alpha_complex/triangle_set/triangles/faces')    
        dataset.load_ion_to_edge_distances(
            file_name='PARAPROBE.Distancer.Results.SimID.' + JOBID + '.nxs',
            dataset_name='/entry/process0/point_to_triangle_set/distance')        
        # no optional filters, we want to analyze the entire dataset

        # define high-throughput job of e.g. multiple delocalization tasks with each creating multiple iso-surfaces
        task = Delocalization()
        task.set_delocalization_input(source='default')
        task.set_delocalization_normalization(method='composition')  # ...with grid values normalized to atomic fraction (at.-%)
        task.set_delocalization_elements(['Y', 'Ti', 'O'])  # iso-surface defined by Y+Ti+O ...
        task.set_delocalization_gridresolutions(length=[1.])  # nm, list of voxel edge length, for each length one analysis
        task.set_delocalization_kernel(sigma=[1.0], size=3)  # nm and pixel respectively
        # task.set_delocalization_isosurfaces(phi=np.linspace(start=0.04, stop=0.04, num=1, endpoint=True))
        task.set_delocalization_isosurfaces(phi=np.linspace(start=0.01, stop=0.21, num=21, endpoint=True)) # isosurface starting at 1 at.-% in steps of 1 at.-% until 21 at.-%
        task.set_delocalization_edge_handling(method='default')
        task.set_delocalization_edge_threshold(EPSILON)
        # nm, with the threshold approaching epsilon only those objects are considered close to the edge which have at least one ion with an ion-to-edge distance almost zero, 
        # as these are only the ions virtually on the triangulated surface mesh representing the edge, virtually each object is considered inside
        # which quantities to process for the iso-surface
        task.report_fields_and_gradients(True)
        task.report_triangle_soup(True)
        task.report_objects(True)
        task.report_objects_properties(True)
        task.report_objects_geometry(True)
        task.report_objects_optimal_bounding_box(True)
        task.report_objects_ions(True)
        task.report_objects_edge_contact(True)
        # in some cases one may desire to export also doppelganger meshes to those objects which do not represent a closed mesh in the first place
        # these doppelganger are the so-called proxy for an object, proxies can be useful to consider objects at the edge of the dataset in
        # coprecipitation analyses where otherwise because of the finite size of the dataset and the open mesh of many objects these objects would be discarded
        task.report_proxies(False)
        task.report_proxies_properties(False)
        task.report_proxies_geometry(False)
        task.report_proxies_optimal_bounding_box(False)
        task.report_proxies_ions(False)
        # task.report_objects_autoproxigrams(False)
        # task.report_objects_autoproxigrams_edge_contact(False)
        
        nanochem.add_task(dataset, task)            
        NANOCHEM_CONFIG[jobid] = nanochem.configure(int(jobid))
    else:
        raise ValueError('Other modes are irrelevant for this example, use isosurface_scanning !')
print(NANOCHEM_CONFIG)

In [ ]:
# inspect config if desired
H5Web(NANOCHEM_CONFIG[jobid])

In [ ]:
# execute paraprobe-nanochem C/C++ tool
NANOCHEM_RESULTS = {}
for jobid in jobids:
    JOBID = str(jobid)
    CONFIG = NANOCHEM_CONFIG[jobid]
    ! export OMP_NUM_THREADS=4 && mpiexec -n 1 ./../../../code/paraprobe_nanochem $jobid $CONFIG 1>{'PARAPROBE.Nanochem.Results.StdoutLog.SimID.' + JOBID + '.txt'} 2>{'PARAPROBE.Nanochem.Results.StderrLog.SimID.' + JOBID + '.txt'}
    NANOCHEM_RESULTS[jobid] = 'PARAPROBE.Nanochem.Results.SimID.' + JOBID + '.nxs'
print(NANOCHEM_RESULTS)

In [ ]:
# inspect paraprobe-nanochem results
print(str(np.around(os.path.getsize(NANOCHEM_RESULTS[jobid])/1024/1024, decimals=3)) + ' MiB')
H5Web(NANOCHEM_RESULTS[jobid])

See how to post-process paraprobe-nanochem results with paraprobe-autoreporter.

In [ ]:
# additional corporate design/preconfigured analyses from paraprobe-autoreporter
NANOCHEM_RESULTS = {}
NANOCHEM_RESULTS[jobid] = 'PARAPROBE.Nanochem.Results.SimID.636502001.nxs'
for jobid in jobids:
    JOBID = str(jobid)
    nanochem_report = AutoReporterNanochem(NANOCHEM_RESULTS[jobid], dataset_id=0)
    nanochem_report.get_delocalization(delocalization_task_id=0)
    isosurface_task_id = 1
    # get the iso-value for a specific iso-surface
    print('isosurface_task_id ' + str(isosurface_task_id))
    print(nanochem_report.delocalization.isosurface_tasks[str(isosurface_task_id)].isovalue)
    for isosurface_task_id in np.arange(9,13):
        print('isosurface_task_id ' + str(isosurface_task_id))
        print(nanochem_report.delocalization.isosurface_tasks[str(isosurface_task_id)].isovalue)
    nanochem_report.get_isosurface_objects_volume_and_number_over_isovalue(delocalization_task_id=0)

In [ ]:
NANOCHEM_PLOT = 'PARAPROBE.Nanochem.Results.SimID.636502001.nxs.DelocTaskId.0.VolOverIsoComposition.png'
NANOCHEM_PLOT = 'PARAPROBE.Nanochem.Results.SimID.636502001.nxs.DelocTaskId.0.NumberOverIsoComposition.png'
Image(filename=NANOCHEM_PLOT, width=500, height=500)

When probing from 1at.-% to 21 at.-% Y+Ti+O iso-composition, the number of objects and their accumulated volume<br>
changes as a function of iso-value. Shown here exemplarily are objects which are located sufficiently inside the<br>
dataset, i.e. $\geq d_{edge}$.<br>

In this example the objects get identified the clearer the larger it is the iso-value. With increasing iso-value<br>
the initial connected iso-surface fragments into pieces and individual objects become eroded towards their interior<br>
which is reduces the objects individually detected volume. Interestingly and by chance in fact, especially the few<br>
large objects were already laying close to the edge of the dataset. Therefore, they qualify as objects with edge contact.<br>
Consequently, they are removed from the analyses NumberOverIsoCompositionCurve. If the reconstructed volume would have<br>
been larger, it would have likely contained more objects which could have stabilized the curve.<br>
In effect, the dataset shows strong differences which are evidence of finite counting effects.<br>

From this we learn again why rigorous analyses of precipitate/object statistics are of particular relevance and<br>
that the same dataset can show very different significance with respect for composition and precipitate analyses.<br>

# 3. Spatial correlation analyses
***

In [ ]:
# configure paraprobe-intersector
INTERSECTOR_CONFIG = {}
for jobid in jobids:
    JOBID = str(jobid)
    
    intersector = ParmsetupIntersector()
    # sequence of iso-surface configurations
   
    for i in np.arange(0, 1):
    # for i in np.arange(0, 3):
    #     # will create three tasks, one for 0 against 1, one for 1 against 2, and one for 2 against 3
        task = VolumeVolumeTask()
        task.set_analyze_proximity(True)
        task.set_analyze_coprecipitation(True)
        task.set_threshold_proximity(2.)  # nm
        
        isrf_curr = int(i)
        cur = VolumeFeatureSet('current', isrf_curr)
        # define up to four named volumetric_feature sub-sets with in the current set, objects far from edge, close to edge, proxies far from edge, proxies close to edge
        h5r = h5py.File('PARAPROBE.Nanochem.Results.SimID.636502001.nxs', 'r')
        grpnm = '/entry/process0/iso_surface_analysis/delocalization0/iso_surface' + str(isrf_curr) + '/triangle_soup/volumetric_feature/objects/objects_far_from_edge'
        ids = h5r[grpnm + '/feature_identifier'][:, 0]
        # print(np.shape(ids))
        # print(ids.dtype)
        s = VolumeFeatureSubSet(
            feature_type='objects_far_from_edge',
            filename='PARAPROBE.Nanochem.Results.SimID.636502001.nxs',
            groupname_geometry_prefix=grpnm, feature_identifier=ids)
        h5r.close()
        cur.add_feature_type(s)
        
        isrf_next = int(i)  # verification, compare objects against themselves
        # isrf_next = int(i + 1)  # sequence of configurations
        nxt = VolumeFeatureSet('next', isrf_next)
        h5r = h5py.File('PARAPROBE.Nanochem.Results.SimID.636502001.nxs', 'r')
        grpnm = '/entry/process0/iso_surface_analysis/delocalization0/iso_surface' + str(isrf_next) + '/triangle_soup/volumetric_feature/objects/objects_far_from_edge'
        ids = h5r[grpnm + '/feature_identifier'][:, 0]
        # print(np.shape(ids))
        # print(ids.dtype)
        s = VolumeFeatureSubSet(
            feature_type='objects_far_from_edge',
            filename='PARAPROBE.Nanochem.Results.SimID.636502001.nxs',
            groupname_geometry_prefix=grpnm, feature_identifier=ids)
        h5r.close()
        nxt.add_feature_type(s)
        
        task.add_feature_sets(cur, nxt)
        intersector.add_task(task)

    INTERSECTOR_CONFIG[jobid] = intersector.configure(int(jobid))  # , verbose=True)
print(INTERSECTOR_CONFIG)

In [ ]:
# inspect config if desired
H5Web(INTERSECTOR_CONFIG[jobid])

In [ ]:
# execute paraprobe-intersector C/C++ tool
INTERSECTOR_RESULTS = {}
for jobid in jobids:
    JOBID = str(jobid)
    CONFIG = INTERSECTOR_CONFIG[jobid]
    ! export OMP_NUM_THREADS=1 && mpiexec -n 1 ./../../../code/paraprobe_intersector $jobid $CONFIG 1>{'PARAPROBE.Intersector.Results.SimID.' + JOBID + '.txt'} 2>{'PARAPROBE.Intersector.Results.StderrLog.SimID.' + JOBID + '.txt'}
    INTERSECTOR_RESULTS[jobid] = 'PARAPROBE.Intersector.Results.SimID.' + JOBID + '.nxs'
print(INTERSECTOR_RESULTS)

In [ ]:
# inspect paraprobe-intersector results
print(str(np.around(os.path.getsize(INTERSECTOR_RESULTS[jobid])/1024/1024, decimals=3)) + ' MiB')
H5Web(INTERSECTOR_RESULTS[jobid])

In this particular example we performed a verification based on the following idea: If we set the current and next set to represent the same set of features,<br>
we expect that all features have a link to themselves because they each feature of course perfectly intersects with itself. However, there is more to it this<br>
story because we also analyze the proximity of nearby objects. Here, specifically the proximity was set to $2nm$ and assuming we compare isosurface0 against<br>
isosurface0). So in addition to links from volumetric intersections, we also expect to find links to nearby neighbors of features; and indeed this is exactly<br>
what we observe in when inspecting */entry/process0/volume_volume_spatial_correlation0/current_to_next_link*<br>
So let\'s now post-process how many links of different type we have:<br>

In [ ]:
h5r = h5py.File('PARAPROBE.Intersector.Results.SimID.636502001.nxs', 'r')
c2n_lnk = h5r['/entry/process0/volume_volume_spatial_correlation0/current_to_next_link'][:, :]
c2n_lnk_type = h5r['/entry/process0/volume_volume_spatial_correlation0/current_to_next_link_type'][:, 0]
print('There are ' + str(np.shape(c2n_lnk_type)[0]) + ' c2n links in total')
print(str(np.sum(c2n_lnk_type == 0)) + ' of these are from volumetric intersections/overlap')
print(str(np.sum(c2n_lnk_type == 1)) + ' of these are from proximity')
h5r.close()

Finally, let\'s confirm that the links for intersections are indeed always self-intersection, i.e. the feature_identifier for the current<br>
and the feature_identifier for the next_set are the same.

In [ ]:
identifier_difference = np.int64(c2n_lnk[:, 0]) - np.int64(c2n_lnk[:, 1])
print(identifier_difference.dtype)
print(np.unique(identifier_difference[c2n_lnk_type == 0]))

Evidently, the feature_identifier difference between the current and the next is 0 for exactly all those rows of the<br>
matrix c2n_lnk where the lnk_type is 0x00 / 0 representing the flag which marks cases of volumetric intersection.

These simple examples of spatial correlation analysis give you a glimpse of the capabilities of these tools.<br>
The analysis of coprecipitation can be explored in substantial more detail using so-called coprecipitation<br>
analysis, i.e. graph-based spatial correlation analysis using operations on mixtures of sets.<br>
You can explore such analyses in <a href="https://doi.org/10.48550/arXiv.2205.13510">our recent work with V. Rielli et al.</a> via its own tutorial.<br>

## Conclusions
***

A typical workflow with the paraprobe-toolbox was exemplified for an ODS steel with Y+Ti+O dispersoids.<br>
Apart from offering guidance on the tools the tutorial shows that:<br>
* Care has to be taken especially for small datasets when it comes to characterizing the number and properties of precipitates.<br>
* The here presented tools enable users to perform these analyses with an automated provenance tracking to align better with the<br>
  aims of the FAIR data stewardship principles. Specifically formatted HDF5 files use NeXus application definitions to document<br>
  each result and keep track of versioned configuration files and versioned datasets to maximize efforts aiming at numerical reproducibility.<br>
* We have also shown how paraprobe-intersector can be used to perform spatial correlation analyses on volumetric features.<br>

### Questions?
***

If you run in problems or have suggestion how we can improve these tools, if you feel you can contribute dataset<br>
to support us with developing further these tools, or if you would like to get support with specific data analyses:<br>
Feel free to contact me directly or members of the FAIRmat team: <a href="https://www.fairmat-nfdi.eu/fairmat/about-fairmat/team-fairmat">M. Kühbach et al.</a>

### References, acknowledgements, funding
***

<a href="https://doi.org/10.48550/arXiv.2205.13510">The preprint to the paper that is associated with this analysis is available here.</a><br>
<a href="https://fairmat-experimental.github.io/nexus-fairmat-proposal">Used NeXus/HDF5 data schemes can be found here.</a><br>
(c) Markus Kühbach, 2022/12<br>

<a href="https://www.fairmat-nfdi.eu/fairmat/">FAIRmat</a> is a consortium on research data management which is part of the German NFDI.<br>
The project is funded by the Deutsche Forschungsgemeinschaft (DFG, German Research Foundation) – project 460197019.